In [5]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import plotly.express as px 
import matplotlib.pyplot as plt 
import streamlit as st
import plotly.io as pio
import pydeck as pdk
df = pd.read_csv("C:\\Users\\nabil\\Downloads\\18NYC23.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 841660 entries, 0 to 841659
Data columns (total 11 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   LATITUDE                  841660 non-null  float64
 1   LONGITUDE                 841660 non-null  float64
 2   TOTAL_INJURIES            841660 non-null  int64  
 3   TOTAL_FATALITIES          841660 non-null  int64  
 4   PEDESTRIANS_INJURIES      841660 non-null  int64  
 5   PEDESTRIANS_FATALITIES    841660 non-null  int64  
 6   MAIN_CONTRIBUTING_FACTOR  841660 non-null  object 
 7   COLLISION_ID              841660 non-null  int64  
 8   DATETIME                  841660 non-null  object 
 9   SUB_VEHICLE_NUM           841660 non-null  int64  
 10  MAIN_VEHICLE_TYPE         841660 non-null  object 
dtypes: float64(2), int64(6), object(3)
memory usage: 70.6+ MB


In [91]:
%%writefile coll.py

import streamlit as st
import pandas as pd
import base64
import folium
from streamlit_folium import folium_static
from geopy.geocoders import Nominatim
import numpy as np
import pydeck as pdk
import plotly.express as px
from folium.plugins import FastMarkerCluster
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt 
import plotly.io as pio
from sklearn.preprocessing import MinMaxScaler


def Input():
    
    # Create a map centered around New York City
    nyc_location = [40.7128, -74.0060]
    m = folium.Map(location=nyc_location, zoom_start=12)
    
    # Define a geocoder to find the address from latitude and longitude
    geolocator = Nominatim(user_agent="my_app")
    
    # Add click event functionality to the map
    def on_map_click(event):
        # Get the latitude and longitude of the clicked location
        lat, lon = event.latlng
    
        # Reverse geocode to get the address
        location = geolocator.reverse(f"{lat}, {lon}")
        address = location.address if location else "Unknown"
    
        # Add a marker to the clicked location with the address as a popup
        folium.Marker([lat, lon], popup=address).add_to(m)
    
        # Display the zip code, address, latitude, and longitude
        st.write(f"Zip Code: {location.raw['address'].get('postcode', 'Unknown')}")
        st.write(f"Address: {address}")
        st.write(f"Latitude: {lat}")
        st.write(f"Longitude: {lon}")
    
    # Register the click event handler
    m.add_child(folium.LatLngPopup())
    m.add_child(folium.ClickForMarker(popup=None))
    
    # Display the map in Streamlit
    st.header("New York City Map")
    folium_static(m)
    




    def main():
        st.title("Data Entry and Download")
        with st.form("data_form"):
            latitude_input = st.text_input("Latitude", value="0.0")
            latitude = float(latitude_input) if latitude_input else 0.0
            longitude_input = st.text_input("Longitude", value="0.0")
            longitude = float(longitude_input) if longitude_input else 0.0
            total_injuries = int(st.number_input("Total Injuries", value=0, min_value=0))
            total_fatalities = int(st.number_input("Total Fatalities", value=0, min_value=0))
            pedestrians_injuries =int( st.number_input("Pedestrians Injuries", value=0, min_value=0))
            pedestrians_fatalities =int( st.number_input("Pedestrians Fatalities", value=0, min_value=0))
            main_contributing_factor = st.text_input("Main Contributing Factor", max_chars=30)
            collision_id = st.number_input("Collision ID",value=0, min_value=0)
            datetime = st.text_input("Date and Time", value="", help="Format: YYYY-MM-DD HH:MM:SS")
            sub_vehicle_number = int(st.number_input("Sub vehicle Number",value=0, min_value=0))
            main_vehicle_type = st.text_input("Main Vehicle Type", max_chars=20)
    
            add_button_clicked = st.form_submit_button("Add")
    
        new_rows = []
    
        if add_button_clicked:
            
            
            
            
            
            if latitude or longitude or total_injuries or total_fatalities or pedestrians_injuries or pedestrians_fatalities or main_contributing_factor or collision_id or datetime or sub_vehicle_number and main_vehicle_type:
                new_row = {
                    "Latitude": latitude,
                    "Longitude": longitude,
                    "Total Injuries": total_injuries,
                    "Total Fatalities": total_fatalities,
                    "Pedestrians Injuries": pedestrians_injuries,
                    "Pedestrians Fatalities": pedestrians_fatalities,
                    "Main Contributing Factor": main_contributing_factor,
                    "Collision ID": collision_id,
                    "Date and Time": datetime,
                    "Sub vehicle Number": sub_vehicle_number,
                    "Main Vehicle Type": main_vehicle_type
                }
                new_rows.append(new_row) 
    
        new_df = pd.DataFrame(new_rows)
    
        if not new_df.empty:
            st.write("New Data:")
            st.dataframe(new_df)
    
            organization_name = "Your Organization Name"
            report = generate_report(new_df, organization_name)
    
            new_data = new_df.to_string(index=False)
    
            st.write("Text Report:")
            st.text(report)
    
            download_button(report, new_data, "report_with_data.txt")
    
    
    def generate_report(df, organization_name):
        report = ""
    
        for index, row in df.iterrows():
            report += f"Accident Report:\n\n"
            report += f"Date and Time: {row['Date and Time']}\n"
            report += f"Location: The accident occurred at the latitude {row['Latitude']} and longitude {row['Longitude']}.\n\n"
            report += f"Details:\n"
            report += f"- Total Injuries: {row['Total Injuries']} people were injured in the accident.\n"
            report += f"- Total Fatalities: {row['Total Fatalities']} fatalities were reported.\n"
            report += f"- Pedestrians Injuries: {row['Pedestrians Injuries']} pedestrians sustained injuries.\n"
            report += f"- Pedestrians Fatalities: {row['Pedestrians Fatalities']} pedestrian fatalities were reported.\n"
            report += f"- Main Contributing Factor: {row['Main Contributing Factor']}\n"
            report += f"- Collision ID: {row['Collision ID']}\n\n"
            report += f"Vehicle Information:\n"
            report += f"- Main Vehicle Type: {row['Main Vehicle Type']}\n"
            report += f"- Sub vehicle Number: {row['Sub vehicle Number']}\n\n"
            report += f"Please note that this report is based on the available information and may be subject to updates or revisions as further investigation is conducted.\n\n"
            report += f"If you have any additional questions or require further assistance, please don't hesitate to contact us. Stay safe on the roads!\n\n"
            report += f"Best regards,\n"
            
    
        return report
    
    
    def download_button(report, data, filename):
        content = f"{report}\n\nNew Data:\n{data}"
        b64 = base64.b64encode(content.encode()).decode()
        href = f"<a href='data:file/txt;base64,{b64}' download='{filename}'>Download {filename}</a>"
        st.markdown(href, unsafe_allow_html=True)
    
    
    if __name__ == "__main__":
        main()
        
        
    
################################################################################################################    
################################################################################################################    
    
    
def Output():    
    # Load your dataset into a DataFrame
    df = pd.read_csv("C:\\Users\\nabil\\Downloads\\18NYC23.csv")
    
    # Create an input box for the user to enter COLLISION_ID
    collision_id = st.text_input("Enter COLLISION_ID")
    
    # Define the generate_report function
    def generate_report(df):
        report = ""
    
        for index, row in df.iterrows():
            report += "Accident Report:\n\n"
            report += f"Date and Time: {row['DATETIME']}\n"
            report += f"Location: The accident occurred at the latitude {row['LATITUDE']} and longitude {row['LONGITUDE']}.\n\n"
            report += "Details:\n"
            report += f"- Total Injuries: {row['TOTAL_INJURIES']} people were injured in the accident.\n"
            report += f"- Total Fatalities: {row['TOTAL_FATALITIES']} fatalities were reported.\n"
            report += f"- Pedestrians Injuries: {row['PEDESTRIANS_INJURIES']} pedestrians sustained injuries.\n"
            report += f"- Pedestrians Fatalities: {row['PEDESTRIANS_FATALITIES']} pedestrian fatalities were reported.\n"
            report += f"- Main Contributing Factor: {row['MAIN_CONTRIBUTING_FACTOR']}\n"
            report += f"- Collision ID: {row['COLLISION_ID']}\n\n"
            report += "Vehicle Information:\n"
            report += f"- Main Vehicle Type: {row['MAIN_VEHICLE_TYPE']}\n"
            report += f"- Sub Vehicle Number: {row['SUB_VEHICLE_NUM']}\n\n"
            report += "Please note that this report is based on the available information and may be subject to updates or revisions as further investigation is conducted.\n\n"
            report += "If you have any additional questions or require further assistance, please don't hesitate to contact us. Stay safe on the roads!\n\n"
            report += "Best regards,\n\n"
    
        return report
    
    # Define the download_button function
    def download_button(report, data, filename):
        content = f"{report}\n\nNew Data:\n{data}"
        b64 = base64.b64encode(content.encode()).decode()
        href = f"<a href='data:file/txt;base64,{b64}' download='{filename}'>Download {filename}</a>"
        st.markdown(href, unsafe_allow_html=True)
    
    # Check if the entered COLLISION_ID exists in the DataFrame
    if collision_id in df['COLLISION_ID'].astype(str).values:
        # Filter the DataFrame based on the entered COLLISION_ID
        filtered_data = df[df['COLLISION_ID'].astype(str) == collision_id]
        # Generate the report for the filtered data
        report = generate_report(filtered_data)
        # Display the report
        st.text(report)
        # Display the filtered data
        st.header("Filtered Data")
        st.dataframe(filtered_data)
        # Create the download button
        download_button(report, filtered_data.to_string(), f"collision_report_{collision_id}.txt")
    else:
        st.write("No data found for the entered COLLISION_ID")    
        
    
pages = {
    'Input Tool': Input,
    'Output Tool': Output
}

select_page = st.sidebar.selectbox('Select overview', pages.keys())
pages[select_page]()     
##########################################################################################################################
##########################################################################################################################




############################################################################################################################
##########################################################################################################################



def Timing():
    # Load data
    @st.cache_data(persist=True)
    def load_data(year):
        df = pd.read_csv("C:\\Users\\nabil\\Downloads\\18NYC23.csv")
        df['DATETIME'] = pd.to_datetime(df['DATETIME'])
        df.dropna(subset=['LATITUDE', 'LONGITUDE'], inplace=True)
        df = df[df['DATETIME'].dt.year == year]
        return df
    
    # Year selection
    selected_year = st.sidebar.selectbox("Choose a year", [2023] + list(range(2018, 2023+1)))  # Custom year 2023 option
    data = load_data(selected_year)
    
    # Hourly Collisions
    st.header("How many collisions occur during a given time of day?")
    hour = st.slider("Hour to look at", 0, 23)
    hourly_data = data[data['DATETIME'].dt.hour == hour]
    st.markdown("Vehicle collisions between %i:00 and %i:00" % (hour, (hour + 1) % 24))
    
    # Display data
    st.subheader("Collisions during the selected hour")
    st.map(hourly_data[['LATITUDE', 'LONGITUDE']])
    st.write(hourly_data[['DATETIME', 'LATITUDE', 'LONGITUDE']])
    
    midpoint = (np.average(hourly_data["LATITUDE"]), np.average(hourly_data["LONGITUDE"]))
    st.write(pdk.Deck(
        map_style="mapbox://styles/mapbox/light-v9",
        initial_view_state={
            "latitude": midpoint[0],
            "longitude": midpoint[1],
            "zoom": 11,
            "pitch": 50,
        },
        layers=[
            pdk.Layer(
            "HexagonLayer",
            data=hourly_data[['DATETIME', 'LATITUDE', 'LONGITUDE']],
            get_position=["LONGITUDE", "LATITUDE"],
            auto_highlight=True,
            radius=100,
            extruded=True,
            pickable=True,
            elevation_scale=4,
            elevation_range=[0, 1000],
            ),
        ],
    ))
    
    # Raw Data
    if st.checkbox("Show raw data", False):
        st.subheader("Raw data by minute between %i:00 and %i:00" % (hour, (hour + 1) % 24))
        st.write(hourly_data)
    
    # Breakdown by Minute
    st.subheader("Breakdown by minute between %i:00 and %i:00" % (hour, (hour + 1) % 24))
    filtered_data = hourly_data[
        (hourly_data['DATETIME'].dt.hour >= hour) & (hourly_data['DATETIME'].dt.hour < (hour + 1))
    ]
    hist = np.histogram(filtered_data['DATETIME'].dt.minute, bins=60, range=(0, 60))[0]
    chart_data = pd.DataFrame({"minute": range(60), "crashes": hist})
    
    fig = px.bar(chart_data, x='minute', y='crashes', hover_data=['minute', 'crashes'], height=400)
    st.write(fig)
    
    
    # Top 5 Dangerous Streets by Affected Class
    
    
show_plots = st.sidebar.checkbox('ACCIDENT_TIMING')
if show_plots:
    Timing()

###########################################################################################################################
###########################################################################################################################





###################################################################################################
###################################################################################################
## Graphs

def GRAPHS():    
    import streamlit as st
    import pandas as pd
    import plotly.graph_objects as go
    import plotly.subplots as sp
    
    df = pd.read_csv("C:\\Users\\nabil\\Downloads\\18NYC23.csv")
    # Convert the 'DATETIME' column to a pandas datetime type
    df['DATETIME'] = pd.to_datetime(df['DATETIME'])
    
    # Extract the year and month from the 'DATETIME' column
    df['YEAR'] = df['DATETIME'].dt.year
    df['MONTH'] = df['DATETIME'].dt.month
    
    # Group the data by year and month and count the number of collisions
    grouped_data = df.groupby(['YEAR', 'MONTH']).size().reset_index(name='collisions_count')
    
    # Create a new column combining the year and month for labeling purposes
    grouped_data['YEAR_MONTH'] = grouped_data['YEAR'].astype(str) + '/' + grouped_data['MONTH'].astype(str)
    
    # Sort the data by year and month
    grouped_data.sort_values(['YEAR', 'MONTH'], inplace=True)
    st.subheader('Plot 1')
    # Plot the line chart
    fig1 = go.Figure(data=go.Scatter(x=grouped_data['YEAR_MONTH'], y=grouped_data['collisions_count'], mode='lines+markers'))
    
    # Set the title and axis labels
    fig1.update_layout(title='Collisions Variation by Year and Month',
                      xaxis_title='Year/Month',
                      yaxis_title='Collisions Count')
    
    # Rotate the x-axis labels for better readability
    fig1.update_layout(xaxis_tickangle=-45)
    
    # Display the chart using Streamlit
    st.plotly_chart(fig1)
    
        ##########################################################################################################
    
    import pandas as pd
    import plotly.graph_objects as go
    import plotly.subplots as sp
     
     # Load the data
    df = pd.read_csv("C:\\Users\\nabil\\Downloads\\18NYC23.csv")
    df['DATETIME'] = pd.to_datetime(df['DATETIME'])  # Convert 'DATETIME' column to datetime if not already
    
    # Extract the year and month from the 'DATETIME' column
    df['YEAR'] = df['DATETIME'].dt.year
    df['MONTH'] = df['DATETIME'].dt.month
    
    # Group the data by year and month and calculate the sum of 'TOTAL_INJURIES' and 'TOTAL_FATALITIES'
    injuries_by_year_month = df.groupby(['YEAR', 'MONTH'])[['TOTAL_INJURIES', 'TOTAL_FATALITIES']].sum().reset_index()
    st.subheader('Plot 2')
    # Create subplots with shared x-axis
    fig2 = sp.make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1)
    
    # Add scatter plot for 'TOTAL_INJURIES'
    fig2.add_trace(go.Scatter(x=injuries_by_year_month['MONTH'], y=injuries_by_year_month['TOTAL_INJURIES'], mode='markers',
                             name='Total Injuries', marker=dict(color='blue', size=8)), row=1, col=1)
    
    # Add bar chart for 'TOTAL_FATALITIES'
    fig2.add_trace(go.Bar(x=injuries_by_year_month['MONTH'], y=injuries_by_year_month['TOTAL_FATALITIES'],
                         name='Total Fatalities', marker_color='red'), row=2, col=1)
    
    # Update layout
    fig2.update_layout(
        title='Total Injuries and Fatalities by Year and Month',
        xaxis_title='Month',
        height=600,  # Adjust the height of the plot
        showlegend=True,  # Show legend
        legend=dict(x=0.02, y=0.98),  # Adjust the position of the legend
        margin=dict(l=30, r=30, t=80, b=30),  # Adjust the margins
    )
    
    # Update y-axis titles
    fig2.update_yaxes(title_text='Total Injuries', row=1, col=1)
    fig2.update_yaxes(title_text='Total Fatalities', row=2, col=1)
    
    # Display the plot in Streamlit
    st.plotly_chart(fig2)
    
    ###################################################################################
    
    
    
    import pandas as pd
    import plotly.graph_objects as go
    df = pd.read_csv("C:\\Users\\nabil\\Downloads\\18NYC23.csv")
    # Assuming you have a DataFrame named 'df' with columns 'MAIN_CONTRIBUTING_FACTOR', 'TOTAL_INJURIES', and 'TOTAL_FATALITIES'
    
    # Calculate the total injuries and fatalities for each contributing factor
    injuries_fatalities_by_factor = df.groupby('MAIN_CONTRIBUTING_FACTOR')[['TOTAL_INJURIES', 'TOTAL_FATALITIES']].sum().reset_index()
    
    # Calculate the percentage of injuries and fatalities for each contributing factor
    injuries_fatalities_by_factor['PERCENTAGE_INJURIES'] = injuries_fatalities_by_factor['TOTAL_INJURIES'] / injuries_fatalities_by_factor['TOTAL_INJURIES'].sum() * 100
    injuries_fatalities_by_factor['PERCENTAGE_FATALITIES'] = injuries_fatalities_by_factor['TOTAL_FATALITIES'] / injuries_fatalities_by_factor['TOTAL_FATALITIES'].sum() * 100
    
    # Sort the DataFrame by the percentage of fatalities in descending order
    injuries_fatalities_by_factor = injuries_fatalities_by_factor.sort_values('PERCENTAGE_FATALITIES', ascending=False)
    
    # Select the top 20 factors
    top_20_factors = injuries_fatalities_by_factor.head(20)
    
    st.subheader('Plot 3')
    fig3 = go.Figure(data=[
        go.Bar(name='Injuries', x=top_20_factors['MAIN_CONTRIBUTING_FACTOR'], y=top_20_factors['PERCENTAGE_INJURIES']),
        go.Bar(name='Fatalities', x=top_20_factors['MAIN_CONTRIBUTING_FACTOR'], y=top_20_factors['PERCENTAGE_FATALITIES'])
    ])
    
    # Update layout
    fig3.update_layout(
        title='Percentage of Injuries and Fatalities by Top 20 Contributing Factors',
        xaxis_title='Contributing Factor',
        yaxis_title='Percentage',
        barmode='stack',
        legend=dict(x=0.02, y=0.98),  # Adjust the position of the legend
        margin=dict(l=30, r=30, t=80, b=30),  # Adjust the margins
        width=1200,  # Adjust the width of the plot
        height=700,  # Adjust the height of the plot
    )
    
    st.plotly_chart(fig3)
    
    
    
    
    grouped_data_injuries = df.groupby('MAIN_VEHICLE_TYPE')['TOTAL_INJURIES'].sum().reset_index(name='total_injuries')
    
    # Calculate the percentage relative to the total injuries for each MAIN_VEHICLE_TYPE
    grouped_data_injuries['percentage'] = (grouped_data_injuries['total_injuries'] / grouped_data_injuries['total_injuries'].sum()) * 100
    
    # Add a line representing the percentage of total injuries
    plt.plot(grouped_data_injuries['MAIN_VEHICLE_TYPE'], grouped_data_injuries['percentage'], color='red', linewidth=2)
    
    # Continue with the remaining code to plot the stacked bar chart
    grouped_data_inj_type = df.groupby('SUB_VEHICLE_NUM')['TOTAL_INJURIES'].sum().reset_index(name='total_injuries')
    grouped_data_inj_type['percentage'] = (grouped_data_inj_type['total_injuries'] / grouped_data_inj_type['total_injuries'].sum()) * 100
    plt.plot(grouped_data_inj_type['SUB_VEHICLE_NUM'], grouped_data_inj_type['percentage'], color='green', linewidth=2)
    
    # Add labels to the lines
    for x, y in zip(grouped_data_injuries['MAIN_VEHICLE_TYPE'], grouped_data_injuries['percentage']):
        plt.text(x, y, f'{y:.2f}', ha='center', va='bottom', color='red')
    
    for x, y in zip(grouped_data_inj_type['SUB_VEHICLE_NUM'], grouped_data_inj_type['percentage']):
        plt.text(x, y, f'{y:.2f}', ha='center', va='bottom', color='green')
    
    # Add a legend
    plt.legend(['Injuries', 'Effected_Sub Vehicles'])
    
    # Set the title and axis labels
    plt.title('Percentage of Injuries by Cause_Vehicle Type')
    plt.xlabel('Cause_Vehicle Type')
    plt.ylabel('Percentage')
    
    # Display the first plot in Streamlit
    st.pyplot(plt)
    
    # Group the data by MAIN_VEHICLE_TYPE, SUB_VEHICLE_NUM, and calculate the sum of TOTAL_FATALITIES
    grouped_data_fatalities = df.groupby(['SUB_VEHICLE_NUM', 'MAIN_VEHICLE_TYPE'])['TOTAL_FATALITIES'].sum().reset_index(name='total_fatalities')
    
    # Calculate the percentage relative to the total fatalities for each MAIN_VEHICLE_TYPE
    grouped_data_fatalities['percentage'] = grouped_data_fatalities.groupby('MAIN_VEHICLE_TYPE', group_keys=False)['total_fatalities'].apply(lambda x: (x / x.sum()) * 100)
    
    # Pivot the data for plotting
    pivot_data = grouped_data_fatalities.pivot(index='MAIN_VEHICLE_TYPE', columns='SUB_VEHICLE_NUM', values='percentage')
    
    # Create the second plot
    plt.figure(figsize=(10, 6))
    ax = pivot_data.plot(kind='bar', stacked=True, ax=plt.gca())
    plt.title('Percentage of MAIN_VEHICLE_TYPE with respect to SUB_VEHICLE_NUM (relative to TOTAL_FATALITIES)')
    plt.xlabel('MAIN_VEHICLE_TYPE')
    plt.ylabel('Percentage')
    plt.legend(title='SUB_VEHICLE_NUM')
    for container in ax.containers:
        for rect in container:
            width = rect.get_width()
            height = rect.get_height()
            x = rect.get_x()
            y = rect.get_y()
            plt.text(x + width / 2, y + height / 2, f'{height:.2f}', ha='center', va='center')
    plt.xticks(rotation=90)
    
    # Display the second plot in Streamlit
    st.pyplot(plt)
    
    # Display additional plots in the sidebar using tabs
    with st.sidebar:
        st.subheader('Plot 1, Plot 2, Plot 3, and Plot 4')
       
       
show_plots = st.sidebar.checkbox('Graphs')
if show_plots:
    GRAPHS()
    
######################################################################################################################    
def Areas():
    tab1, tab2 = st.tabs(['Accident_Loc','Most_Danger'])
    with tab1:
        st.header("number of collisions in borough")
        def create_map(year):
            df = pd.read_csv("C:\\Users\\nabil\\Downloads\\18NYC23.csv")
            df['DATETIME'] = pd.to_datetime(df['DATETIME']).dt.strftime('%Y-%m-%d %H:%M:%S')
        
            filtered_df = df.loc[df['DATETIME'].str[:4] == str(year), ['LATITUDE', 'LONGITUDE']]
        
            latitude = filtered_df["LATITUDE"].mean()
            longitude = filtered_df["LONGITUDE"].mean()
        
            # Create a base map
            base_map = folium.Map(location=[latitude, longitude], zoom_start=10)
        
            # Create a FastMarkerCluster layer
            marker_cluster = FastMarkerCluster(filtered_df[['LATITUDE', 'LONGITUDE']].values).add_to(base_map)
        
            # Save the map as an HTML file
            base_map.save("map.html")
        
            # Display the map using Streamlit
            st.markdown('<h3>Map</h3>', unsafe_allow_html=True)
            with open("map.html", "r") as f:
                st.components.v1.html(f.read(), height=500)
        
        
        # Streamlit app code
        st.title("Collision Map")
        year = st.number_input("Enter a year:", min_value=2018, max_value=2023, value=2023)
        create_map(year)
        
        
        
    with tab2:

        df = pd.read_csv("C:\\Users\\nabil\\Downloads\\18NYC23.csv")
        df['DANGER_RATE'] = df['TOTAL_INJURIES'] + 2 * df['TOTAL_FATALITIES'] + df['PEDESTRIANS_INJURIES'] + df['PEDESTRIANS_FATALITIES'] + 0.5 * df['SUB_VEHICLE_NUM']
        # Normalize the 'DANGER_RATE' values to a range of 1 to 1000
        scaler = MinMaxScaler(feature_range=(1, 100))
        df['DANGER_RATE'] = scaler.fit_transform(df[['DANGER_RATE']])
        
        
     
        st.header("Top 100 dangerous streets by affected class")
        
        # Sort the DataFrame by 'DANGER_RATE' in descending order and select the top 1000 locations
        top_100_locations = df.nlargest(100, 'DANGER_RATE')
        
        # Reverse geocode the latitude and longitude coordinates to obtain location names
        geolocator = Nominatim(user_agent="my_geocoder")
        locations = []
        for lat, lon in zip(top_100_locations['LATITUDE'], top_100_locations['LONGITUDE']):
            location = geolocator.reverse(f"{lat}, {lon}")
            locations.append(location.address if location else "Unknown")
            
        # Add the location names as a new column in the DataFrame
        top_100_locations['LOCATION'] = locations
        
        # Create the scatter plot
        fig = px.scatter_mapbox(
            top_100_locations,
            lat='LATITUDE',
            lon='LONGITUDE',
            hover_name='LOCATION',
            size='COLLISION_ID',
            color='DANGER_RATE',
            color_continuous_scale='Viridis',
            size_max=10,
            zoom=1
        )
        
        # Update the mapbox style
        fig.update_layout(mapbox_style='carto-positron')
        
        # Update the layout of the plot
        fig.update_layout(
            title='Top 100 danger Locations in NYC',
            margin=dict(l=0, r=0, t=40, b=0)
        )
        
        # Show the scatter plot
        fig.show()
        


show_plots = st.sidebar.checkbox('ACCIDENT-AREAS')
if show_plots:
    Areas()
   
    
    
    

Overwriting coll.py


In [ ]:
! streamlit run coll.py

In [1]:
pip install streamlit_folium

     -------------------------------------- 449.0/449.0 kB 1.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install streamlit-leaflet

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install streamlit folium

Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install streamlit folium geopy

Note: you may need to restart the kernel to use updated packages.
